In [ ]:
import pandas as pd
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential

import sys
sys.path.append('../Modeling')
from Model import NNRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer


In [2]:
random_state = 42
train_test_vali_split = [0.8, 0.2]

In [3]:
data = pd.data = (
    pd.read_csv('../data/energydata_complete.csv')
    .drop(['date', 'rv1', 'rv2'], axis=1)
    .assign(target = lambda k: k.lights + k.Appliances)
    .drop(['lights', 'Appliances'], axis=1)
)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('target', axis=1),
    data.target,
    test_size=1 - train_test_vali_split[0],
    random_state=random_state
)

In [5]:
# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Grid Search configuration

In [6]:
def create_model(lr=1e-3, epsilon=1e-8):
    "Function to be passed to GridSearchCV"
    model = NNRegressor(activation="tanh", input_shape=500)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr, epsilon=epsilon),
        loss='mse'
    )
    return model

In [7]:
def create_model(lr=1e-3, epsilon=1e-8):
    "Function to be passed to GridSearchCV"
    model = NNRegressor(activation="tanh", input_shape=500)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr, epsilon=epsilon),
        loss='mse'
    )
    return model

In [8]:
model_search = KerasRegressor(build_fn=create_model, epochs=400)

/tmp/ipykernel_1305/1415693073.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_search = KerasRegressor(build_fn=create_model, epochs=400)


In [9]:
param_grid = {
    'lr': [1e-4, 1e-3, 1e-2],
    'epsilon': [1e-8, 1e-7, 1e-6]
}

In [22]:
selection_score = make_scorer(r2_score)

In [23]:
grid = GridSearchCV(
    estimator=model_search,
    param_grid=param_grid,
    cv=2,
    return_train_score=True,
    scoring=selection_score,
    error_score='raise',
)

In [24]:
grid_result = grid.fit(X_train, y_train.values.reshape(-1))

Epoch 1/400


247/247 [==============================] - 1s 898us/step - loss: 20948.2402
Epoch 2/400
247/247 [==============================] - 0s 941us/step - loss: 20714.2246
Epoch 3/400
247/247 [==============================] - 0s 859us/step - loss: 20337.1191
Epoch 4/400
247/247 [==============================] - 0s 955us/step - loss: 19759.1719
Epoch 5/400
247/247 [==============================] - 0s 859us/step - loss: 18983.5254
Epoch 6/400
247/247 [==============================] - 0s 847us/step - loss: 18049.9434
Epoch 7/400
247/247 [==============================] - 0s 857us/step - loss: 17022.9434
Epoch 8/400
247/247 [==============================] - 0s 873us/step - loss: 15967.4648
Epoch 9/400
247/247 [==============================] - 0s 839us/step - loss: 14924.7539
Epoch 10/400
247/247 [==============================] - 0s 834us/step - loss: 13937.7285
Epoch 11/400
247/247 [==============================] - 0s 829us/step - loss: 13034.6572
Epoch 12/400
247/247 [====================

In [25]:
# r2
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.2f}, std={stdev:.4} using {param}')


 mean=0.21, std=0.001898 using {'epsilon': 1e-08, 'lr': 0.0001}
 mean=0.42, std=0.001537 using {'epsilon': 1e-08, 'lr': 0.001}
 mean=0.34, std=0.008419 using {'epsilon': 1e-08, 'lr': 0.01}
 mean=0.21, std=0.002443 using {'epsilon': 1e-07, 'lr': 0.0001}
 mean=0.42, std=0.003956 using {'epsilon': 1e-07, 'lr': 0.001}
 mean=0.35, std=0.01947 using {'epsilon': 1e-07, 'lr': 0.01}
 mean=0.21, std=0.00185 using {'epsilon': 1e-06, 'lr': 0.0001}
 mean=0.42, std=0.002816 using {'epsilon': 1e-06, 'lr': 0.001}
 mean=0.35, std=0.003568 using {'epsilon': 1e-06, 'lr': 0.01}


In [21]:
# mean square error
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.2f}, std={stdev:.4} using {param}')


 mean=-8663.86, std=223.3 using {'epsilon': 1e-08, 'lr': 0.0001}
 mean=-6415.83, std=116.1 using {'epsilon': 1e-08, 'lr': 0.001}
 mean=-7250.48, std=84.01 using {'epsilon': 1e-08, 'lr': 0.01}
 mean=-8657.54, std=226.3 using {'epsilon': 1e-07, 'lr': 0.0001}
 mean=-6395.18, std=167.7 using {'epsilon': 1e-07, 'lr': 0.001}
 mean=-7148.44, std=132.4 using {'epsilon': 1e-07, 'lr': 0.01}
 mean=-8659.28, std=218.5 using {'epsilon': 1e-06, 'lr': 0.0001}
 mean=-6395.20, std=176.8 using {'epsilon': 1e-06, 'lr': 0.001}
 mean=-7103.33, std=75.02 using {'epsilon': 1e-06, 'lr': 0.01}
